In [7]:
pip install pyspark

In [1]:
from pyspark import SparkConf, SparkContext
from itertools import permutations
import collections
import csv
import time

conf = SparkConf().setMaster('local').setAppName('Apriori')
sc = SparkContext.getOrCreate(conf = conf)


In [3]:
file = sc.textFile('store_data.csv')
start_time = time.time()
#files = file.map(lambda line: line.split(', '))
files = file.mapPartitions(lambda line: csv.reader(line))
#print(files.collect())
items = file.flatMap(lambda line: line.split(','))
#print(items.collect())
uniItems = items.distinct()
wrds = items.map(lambda word: (word, 1))
rdd = wrds.reduceByKey(lambda x,y: x+y)
supportValues = rdd.map(lambda value: value[1]) #supports
minSupValues = 100
supportRdd = rdd.filter(lambda x: x[1] >= minSupValues)
baseRdd = supportRdd.map(lambda x: ([x[0]], x[1]))
print('Created table.')

supportRdd = supportRdd.map(lambda x: x[0])
rddCart = supportRdd

def removeDuplicatedItems(item):
  if (isinstance(item[0], tuple)):
    x = item[0]
    y = item[1]
  else:
    x = [item[0]]
    y = item[1]
  if (any(i == y for i in x) == False):
    temp = list(x)
    temp.append(y)
    temp.sort()
    result = tuple(temp)
    return result
  else:
    return x

combination_length = 2
while (supportRdd.isEmpty() == False):
  combined = supportRdd.cartesian(uniItems)
  combined = combined.map(lambda item: removeDuplicatedItems(item))

  combined = combined.filter(lambda item: len(item) == combination_length)
  combined = combined.distinct()

  combined2 = combined.cartesian(files)
  combined2 = combined2.filter(lambda item: all(x in item[1] for x in item[0]))

  combined2 = combined2.map(lambda item: item[0])
  combined2 = combined2.map(lambda item: (item, 1))
  combined2 = combined2.reduceByKey(lambda x,y: x+y)
  combined2 = combined2.filter(lambda item: item[1] >= minSupValues)

  baseRdd = baseRdd.union(combined2)
  combined2 = combined2.map(lambda item: item[0])
  supportRdd = combined2
  
  combination_length +=1

  print(str(combination_length-1) +' table has created')

class Filter():
  def __init__(self):
    self.stages = 1
  
  def calculateItemSet(self, a, b):
    if (len(a) > len(b)):
      return all(any(i == y for i in a) for y in b)
    else:
      return all(any(i == y for i in b) for y in a)

  def filterForConf(self, item, total):
    if (len(item[0][0]) > len(item[1][0])):
      if (self.calculateItemSet(item[0][0], item[1][0]) == False):
        pass
      else:
        return (item)
    else:
      pass
    self.stages = self.stages + 1

  def calculateConfidence(self, item):
    parent = set(item[0][0])

    if (isinstance(item[1][0], str)):
      child = set([item[1][0]])
    else:
      child = set(item[1][0])

    pSup = item[0][1]
    chSup = item[1][1]
    sup = (pSup / chSup) 
    return list([list(child), list(parent.difference(child)), sup])

calculatedItems = baseRdd.cartesian(baseRdd)
fo = Filter()
total = calculatedItems.count()
rddConfidence = calculatedItems.filter(lambda item: fo.filterForConf(item, total))
rddConfidence = rddConfidence.map(lambda item: fo.calculateConfidence(item))
rddConfidence = rddConfidence.filter(lambda item: item[-1] >= 0.3)


print(rddConfidence.collect())
end_time = time.time()
print('Thời gian thực hiện: ', end_time-start_time)

Created table.
2 table has created
3 table has created
4 table has created
[[['shrimp'], ['mineral water'], 0.3302238805970149], [['low fat yogurt'], ['mineral water'], 0.313588850174216], [['honey'], ['mineral water'], 0.31741573033707865], [['salmon'], ['mineral water'], 0.4012539184952978], [['frozen smoothie'], ['mineral water'], 0.32], [['olive oil'], ['mineral water'], 0.4190283400809717], [['burgers'], ['eggs'], 0.3302752293577982], [['milk'], ['mineral water'], 0.37037037037037035], [['whole wheat rice'], ['mineral water'], 0.3439635535307517], [['turkey'], ['mineral water'], 0.3070362473347548], [['soup'], ['mineral water'], 0.45646437994722955], [['salmon'], ['spaghetti'], 0.3166144200626959], [['olive oil'], ['spaghetti'], 0.3481781376518219], [['cooking oil'], ['mineral water'], 0.39425587467362927], [['turkey'], ['eggs'], 0.31130063965884863], [['frozen vegetables'], ['mineral water'], 0.3748251748251748], [['spaghetti'], ['mineral water'], 0.3430321592649311], [['chocolat

In [4]:
import pandas
data = rddConfidence.collect()

df = pandas.DataFrame(data = data, columns=["Before", "After", "Confidence"])
print(df)

                          Before            After  Confidence
0                       [shrimp]  [mineral water]    0.330224
1               [low fat yogurt]  [mineral water]    0.313589
2                        [honey]  [mineral water]    0.317416
3                       [salmon]  [mineral water]    0.401254
4              [frozen smoothie]  [mineral water]    0.320000
5                    [olive oil]  [mineral water]    0.419028
6                      [burgers]           [eggs]    0.330275
7                         [milk]  [mineral water]    0.370370
8             [whole wheat rice]  [mineral water]    0.343964
9                       [turkey]  [mineral water]    0.307036
10                        [soup]  [mineral water]    0.456464
11                      [salmon]      [spaghetti]    0.316614
12                   [olive oil]      [spaghetti]    0.348178
13                 [cooking oil]  [mineral water]    0.394256
14                      [turkey]           [eggs]    0.311301
15      